In [1]:
import os 
import speech_recognition as sr 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from flask import Flask, request, jsonify
from flask_cors import CORS
from pydub import AudioSegment  
from transformers import pipeline
import spacy
import moviepy.editor as mp 
from moviepy.editor import VideoFileClip  
from googletrans import Translator
from gensim.summarization import summarize
from openai import OpenAI
import math
import re
import subprocess
import string
import os
from huggingface_hub import hf_hub_download
from dotenv import load_dotenv
import pysrt
import requests
import torch
import textrazor

os.environ["CUDA_VISIBLE_DEVICES"]="0"
torch.set_default_device("cuda")

load_dotenv() 
HUGGING_FACE_API_KEY = os.environ.get("HUGGING_FACE_API_KEY")
TEXT_RAZOR_API_KEY = os.environ.get("TEXT_RAZOR_API_KEY")

c:\Users\haris\miniconda3\envs\forPyTorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
prompt = "hello again in this video we will use speech recognition module to extract text from a voice recorded in a MP3 file the main steps for these are first we need to install speech recognition module then if you have a Windows 10 machine with Python 3.8 like me you have also to install between module and with between you install this we can import and initialise the engine with MP3 file and finally extract and print the text first we need to install speech recognition so install speech recognition it enter after this we need to install because I am using Windows 10 with Python 3.80 install NTC seats with me close the terminal next we need to import speech underscore recognition AS or after this we want to initialise the engines so engine is equals to SR dot recognizer the next part is reached our MP3 file let's go a little up because I have here a function that I created and this function generates using pyttsx3 after generating the file it will return the name of that found is equals to generate MP3 file name the source then we want to say something to the end user so we print string that the file is being analysed after painting dogs record and we provide the source now let's move on to the final part let me go a little down let's put here I try and except block and let's declare a text that is equals to engine parts recognise underscore Google and we provide you after this let's print in the console our text and also we want to save in txt file so open we provide the name text from MP3 txt and next we want to write the text in our new files and we provides the text going to exception block here and and don't forget that you can download the source code also don't forget to give these video like share it with your friends subscribe and activate the bell button to get notifications about new videos that come out and as always thank you for watching"

In [17]:
summarizer = pipeline("summarization")
text = summarize(prompt, max_length=130, min_length=30, do_sample=False)
print(text)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
config.json: 100%|██████████| 1.80k/1.80k [00:00<?, ?B/s]
pytorch_model.bin:  35%|███▌      | 430M/1.22G [02:58<07:11, 1.83MB/s] Error while downloading from https://cdn-lfs.huggingface.co/sshleifer/distilbart-cnn-12-6/3bac65d18c99463302d12ca75c2220ea714f9c81ce235f205fa818efe71df6ea?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1714311376&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDMxMTM3Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9zc2hsZWlmZXIvZGlzdGlsYmFydC1jbm4tMTItNi8zYmFjNjVkMThjOTk0NjMzMDJkMTJjYTc1YzIyMjBlYTcxNGY5YzgxY2UyMzVmMjA1ZmE4MThlZmU3MWRmN

In [4]:
textrazor.api_key = TEXT_RAZOR_API_KEY

client = textrazor.TextRazor(extractors=["entities", "topics"])
response = client.analyze(prompt)

for entity in response.entities():
    print(entity.id, entity.relevance_score, entity.confidence_score, entity.freebase_types)

Engine 0.1235 1.095 ['/book/book_subject']
Google 0.2969 12.17 ['/organization/organization_member', '/influence/influence_node', '/business/consumer_company', '/computer/operating_system_developer', '/conferences/conference_sponsor', '/internet/website_owner', '/venture_capital/venture_funded_company', '/business/brand', '/dataworld/data_provider', '/computer/computer_manufacturer_brand', '/education/educational_institution', '/travel/hotel_grading_authority', '/venture_capital/venture_investor', '/business/business_operation', '/law/litigant', '/organization/organization_founder', '/book/book_subject', '/computer/software_developer', '/award/ranked_item', '/organization/organization', '/business/sponsor', '/award/award_presenting_organization', '/award/award_winner', '/business/customer', '/organization/organization_partnership', '/business/issuer', '/law/patent_assignee', '/film/film_subject', '/conferences/conference_subject', '/computer/programming_language_developer', '/award/awa

In [16]:
print(response.topics())

[TextRazor Topic 0 with label Computing platforms, TextRazor Topic 1 with label Digital media, TextRazor Topic 2 with label System software, TextRazor Topic 3 with label Computer engineering, TextRazor Topic 4 with label Computer programming, TextRazor Topic 5 with label Computers, TextRazor Topic 6 with label Computer science, TextRazor Topic 7 with label Software engineering, TextRazor Topic 8 with label Computing, TextRazor Topic 9 with label Software, TextRazor Topic 10 with label Technology, TextRazor Topic 11 with label Operating system families, TextRazor Topic 12 with label Computer architecture, TextRazor Topic 13 with label Information technology, TextRazor Topic 14 with label Windows 10, TextRazor Topic 15 with label Text file, TextRazor Topic 16 with label Digital technology, TextRazor Topic 17 with label MP3, TextRazor Topic 18 with label Microsoft Windows, TextRazor Topic 19 with label Software development, TextRazor Topic 20 with label Office equipment, TextRazor Topic 2

Summary:



In [ ]:
def convert_srt_to_string(srt_file_path): 
    subs = pysrt.open(srt_file_path) 
    converted_content = '' 
    finstr = ''
    for sub in subs:  
        finstr += sub.text + ' '
        start_time = sub.start.to_time().strftime("%H:%M:%S,%f")[:-3]
        end_time = sub.end.to_time().strftime("%H:%M:%S,%f")[:-3] 
        converted_content += f"\n<strong>[{start_time} --> {end_time}]</strong>: {sub.text}\n"

    return converted_content.strip(), finstr

In [ ]:
app = Flask(__name__)
CORS(app, origins="http://localhost:3000", supports_credentials=True)

nlp = spacy.load("en_core_web_sm")

@app.route('/api/analyze-audio/<string:text>', methods=['POST'])
def analyze_text(text):
    if len(text) == 0:
        return jsonify({'error': 'No text provided'}), 400
    
    doc = nlp(text) 
    person_count = len([ent.text for ent in doc.ents if ent.label_ == "PERSON"]) 
    topics = [token.text for token in doc if token.is_alpha and not token.is_stop]

    name = request.form['name']
    file = request.files['file']

    name = name.strip()
    name = name.replace(" ", "_")

    directory_path = os.path.join('uploads', name) 
    filesname, extension = os.path.splitext(file.filename)
    filesname = filesname.strip()
    filesname = filesname.replace(" ", "_")
    text_path = os.path.join(directory_path, "{}.txt".format(filesname))  

    with open(text_path, 'a') as fil:
        fil.write("\nPerson Count: {}\n".format(str(person_count + 1)))
   
    return jsonify({'person_count' : person_count + 1 , 'topic' : topics[0]})

@app.route('/api/convert-video-to-mp3', methods=['POST'])
def convert_video_to_mp3():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    video_file = request.files['file']
    name = request.form['name']
    name = name.strip()
    name = name.replace(" ", "_")

    if video_file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if video_file: 
        try:  
            print(video_file.filename)
            directory_path = os.path.join('uploads', name)
            os.makedirs(directory_path, exist_ok=True) 
            video_path = os.path.join(directory_path, video_file.filename) 
            filesname, extension = os.path.splitext(video_file.filename)
            audio_path = os.path.join(directory_path, "{}.mp3".format(filesname)) 
            video_file.save(video_path)
            video_clip = VideoFileClip(video_path)
            audio_clip = video_clip.audio
            audio_clip.write_audiofile(audio_path)
            audio_clip.close()
            video_clip.close()

            return jsonify({'message': 'Video converted to MP3 successfully', 'audio_file': audio_path}), 200
        except Exception as e:
            return jsonify({'error': 'Conversion error', 'details': str(e)}), 500

@app.route('/api/convert-mp3-to-text', methods=['POST'])
def convert_mp3_to_text():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file'] 
    name = request.form['name']
     
    name = name.strip()
    name = name.replace(" ", "_")

    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:  
        filesname, extension = os.path.splitext(file.filename) 
        filesname = filesname.strip()
        filesname = filesname.replace(" ", "_")

        full_filename = filesname+extension
        directory_path = os.path.join('uploads', name)
        os.makedirs(directory_path, exist_ok=True) 
        mp3_filename = os.path.join(directory_path, full_filename)

        file.save(mp3_filename) 
        segment_dir = 'segments'  
        os.makedirs(segment_dir, exist_ok=True)  
        
        #-----------------------------------------------------------------------------------------
        command1 = f"ffmpeg -i ./uploads/{name}/{filesname}.mp3 -ar 16000 -ac 1 -c:a pcm_s16le ./uploads/{name}/final.wav"  
        command2 = f".\\WHISPER_EXE_FILES\\main.exe -f ./uploads/{name}/final.wav -m .\\WHISPER_EXE_FILES\\ggml-base.en.bin -osrt -t 8 --language auto" 
        
        result = subprocess.run(command1, shell=True, capture_output=True, text=True)
        result2 = subprocess.run(command2, shell=True, capture_output=True, text=True)  

        srt_file_path = f'./uploads/{name}/final.wav.srt'
        text_path = os.path.join(directory_path, "{}.txt".format(filesname)) 
        result_string, joined_text = convert_srt_to_string(srt_file_path)
        os.remove(f"./uploads/{name}/final.wav")

        result_string = result_string.replace("\n", "<br>")

        with open(text_path, 'w') as fil:
            fil.write("\nConverted Text: {}\n".format(str(joined_text)))
 
        return jsonify({'text': joined_text, 'whisper_str': result_string})

@app.route('/api/convert-mp4-to-text', methods=['POST'])
def convert_mp4_to_text():
    language = request.form['language']
    file = request.files['file']
    name = request.form['name']
    
    name = name.strip()
    name = name.replace(" ", "_")

    filesname, extension = os.path.splitext(file.filename)
    filesname = filesname.strip()
    filesname = filesname.replace(" ", "_")

    file = os.path.abspath('./uploads/{}/{}.mp3'.format(name, filesname)) 
    directory_path = os.path.join('uploads', name)
    os.makedirs(directory_path, exist_ok=True)
    if file: 
        mp3_filename = file   
        command1 = f"ffmpeg -i ./uploads/{name}/{filesname}.mp3 -ar 16000 -ac 1 -c:a pcm_s16le ./uploads/{name}/final.wav"  
        result = subprocess.run(command1, shell=True, capture_output=True, text=True)

    segment_dir = 'segments'
    try:
        if not os.path.exists(segment_dir):
            os.mkdir(segment_dir)
    except Exception as e:
        print(f"Error creating directory: {e}")


    command2 = f".\\WHISPER_EXE_FILES\\main.exe -f ./uploads/{name}/final.wav -m .\\WHISPER_EXE_FILES\\ggml-base.en.bin -osrt -t 8 --language auto" 
    result2 = subprocess.run(command2, shell=True, capture_output=True, text=True)  

    srt_file_path = f'./uploads/{name}/final.wav.srt'
    text_path = os.path.join(directory_path, "{}.txt".format(filesname)) 
    result_string, joined_text = convert_srt_to_string(srt_file_path)
    os.remove(f"./uploads/{name}/final.wav")
    result_string = result_string.replace("\n", "<br>")

    with open(text_path, 'w') as fil:
        fil.write("\nConverted Text: {}\n".format(str(joined_text)))

    return jsonify({'text': joined_text, 'whisper_str': result_string})

@app.route('/api/translate_toar/<string:text>', methods=['GET'], endpoint='translate_to_ar')
def translate_to_ar(text):    
    translator = Translator() 
    arabic_translation = translator.translate(text,  src='auto', dest='ar').text
    return jsonify({'translated_txt': arabic_translation})

@app.route('/api/translate_totr/<string:text>', methods=['GET'], endpoint='translate_to_tr')
def translate_to_tr(text):  
    translator = Translator()    

    turkish_translation = translator.translate(text, src='auto', dest='tr').text
    return jsonify({'translated_txt': turkish_translation})

@app.route('/api/translate_toen/<string:text>', methods=['GET'], endpoint='translate_to_en')
def translate_to_en(text):  
    translator = Translator()    

    english_translation = translator.translate(text, src='auto', dest='en').text
    return jsonify({'translated_txt': english_translation})

@app.route('/api/translate_tohi/<string:text>', methods=['GET'], endpoint='translate_to_hi')
def translate_to_hi(text):  
    translator = Translator()    

    hindi_translation = translator.translate(text, src='auto', dest='hi').text
    return jsonify({'translated_txt': hindi_translation})

def add_newlines_every_n_words(input_string, n=10):
    words = input_string.split()
    output_string = ''
    for i, word in enumerate(words):
        if i > 0 and i % n == 0:
            output_string += '\n'
        output_string += word + ' '
    return output_string.strip()

@app.route('/api/findtopic/<string:text>', methods=['POST'])
def topic_finder(text):
    name = request.form['name']
    file = request.files['file']

    name = name.strip()
    name = name.replace(" ", "_")

    pipe = pipeline("text-classification", model="unitary/toxic-bert")
    pipe2 = pipeline("summarization", model="google/pegasus-xsum")

    topic = pipe(text)  
    topic = topic[0]['label']

    topic2 = pipe2(text, max_length = 20)
    topic2 = topic2[0]['summary_text']

    directory_path = os.path.join('uploads', name) 
    filesname, extension = os.path.splitext(file.filename)
    filesname = filesname.strip()
    filesname = filesname.replace(" ", "_")
    text_path = os.path.join(directory_path, "{}.txt".format(filesname))  

    with open(text_path, 'a') as fil:
        fil.write("\nTopic: {}\n".format(topic2))
    
    return jsonify({'topic': topic, 'topic2': topic2})

@app.route('/api/findSummary', methods=['POST'])
def summary_find():
    text = request.form['text']
    pipe = pipeline("summarization", model="google/pegasus-xsum")
    
    temp = add_newlines_every_n_words(text, 30)
    try: 
        summary = summarize(temp)
        print("len: ", len(summary))
    except:
        print("Error in summarization using gensim") 
        summary = ""

    if len(summary) == 0:
        output = pipe(text) 
        summary = output[0]['summary_text']
             
    translator = Translator() 
    arabic_summary = translator.translate(summary, src='en', dest='ar').text
    turkish_summary = translator.translate(summary, src='en', dest='tr').text

    name = request.form['name']
    file = request.files['file']

    name = name.strip()
    name = name.replace(" ", "_")

    directory_path = os.path.join('uploads', name) 
    filesname, extension = os.path.splitext(file.filename)
    filesname = filesname.strip()
    filesname = filesname.replace(" ", "_")
    text_path = os.path.join(directory_path, "{}.txt".format(filesname))  

    with open(text_path, 'a') as fil:
        fil.write("\nSummary: {}\n".format(str(summary)))

    return jsonify({'summary_en': str(summary), 'summary_ar': arabic_summary, 'summary_tr': turkish_summary})

@app.route('/api/sentiment/<string:text>', methods=['POST'])
def sentiment(text):
    name = request.form['name']
    file = request.files['file']

    name = name.strip()
    name = name.replace(" ", "_")

    nltk.download('vader_lexicon')
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    positive_percent = round(scores['pos'] * 100, 2)
    negative_percent = round(scores['neg'] * 100, 2)

    total = positive_percent + negative_percent
    if (total == 0):
        fin_pos = 50
        fin_neg = 50
    else:
        fin_pos = round((positive_percent/total)*100, 2)
        fin_neg = round((negative_percent/total)*100, 2) 


    directory_path = os.path.join('uploads', name) 
    filesname, extension = os.path.splitext(file.filename)
    filesname = filesname.strip()
    filesname = filesname.replace(" ", "_")
    text_path = os.path.join(directory_path, "{}.txt".format(filesname))  

    with open(text_path, 'a') as fil:
        fil.write("\nPositive Sentiment: {}% \nNegative Sentiment: {}%".format(fin_pos, fin_neg))

    return jsonify({'positive': fin_pos, 'negative': fin_neg})

if __name__ == '__main__': 
    app.run(debug=False)